In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [9]:
%cd '/content/drive/My Drive/IEEE BigMM/IEEE-BigMM'
%ls
%cd 'Code/'

/content/drive/.shortcut-targets-by-id/1wOje_Q9CmqTKviMB46IyoCb-GEO1cmWw/IEEE BigMM/IEEE-BigMM
 Code/   Data/   README.md  'Stats .txt'   temp.txt
/content/drive/.shortcut-targets-by-id/1wOje_Q9CmqTKviMB46IyoCb-GEO1cmWw/IEEE BigMM/IEEE-BigMM/Code


In [2]:
import numpy as np 
import pandas as pd 

In [12]:
%ls

Bert_sarcasmv2colab.ipynb  Data_clean.ipynb   Sarcasm/
clean_datav2.csv           final_dataset.csv  Support_Using_ML_Classifier.ipynb


In [13]:
data = pd.read_csv("clean_datav2.csv")

In [14]:
data.head()

,Unnamed: 0,Unnamed: 0.1,ID,tweet_id,text,missing_text,Text_Only_Informative,Image_Only_Informative,Directed_Hate,Generalized_Hate,Sarcasm,Allegation,Justification,Refutation,Support,Oppose
0,0,0,0,1.052240e+18,"['new', 'post', 'domestic', 'violence', 'aware...",0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,1,1,1,1.052210e+18,"['domestic', 'violence', 'awareness', 'caught'...",0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2,2,2,1.052180e+18,"['mother', 'nature', 'metoo', '']",0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,3,3,1.052160e+18,"['ption', '']",0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,4,4,4,1.052100e+18,"['high', 'time', 'metoo', 'named', 'shamed', '...",0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [15]:
import tensorflow as tf
import sklearn
import seaborn as sbs
import sklearn.naive_bayes 
import sklearn.model_selection
import sklearn.metrics

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [16]:
!pip3 install bert-for-tf2

     |████████████████████████████████| 40kB 1.6MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.4-cp36-none-any.whl size=30114 sha256=6893f3759c4dfc957acdeed714de8dea9ccc278c7521f1b6312a18fb8f2c24d4
  Stored in directory: /root/.cache/pip/wheels/cf/3f/4d/79d7735015a5f523648df90d871ce8e89a7df8185f7703eeab
  Created wheel for py-params: filename=py_params-0.9.7-cp36-none-any.whl size=7302 sha256=19a573b3816e8ec68bcfb1125cdb91104a941f3b822354072ceac3edd4d8309e
  Stored in directory: /root/.cache/pip/wheels/67/f5/19/b461849a50aefdf4bab47c4756596e82ee2118b8278e5a1980
  Created wheel for params-flow: filename=params_flow-0.8.2-cp36-none-any.whl size=19473 sha256=dc725d15ae5512388c4b0cdd1b8e2f27f8588cd921dbdc1c56e848ffe9f75158
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow


In [17]:
import tensorflow_hub as hub 
from bert import bert_tokenization
module_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2'
bert_layer = hub.KerasLayer(module_url, trainable=True)

In [18]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = bert_tokenization.FullTokenizer(vocab_file, do_lower_case)

In [19]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence) + [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

def build_model(bert_layer, max_len=512):
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    net = tf.keras.layers.Dense(64, activation='relu')(clf_output)
    net = tf.keras.layers.Dropout(0.2)(net)
    net = tf.keras.layers.Dense(32, activation='relu')(net)
    net = tf.keras.layers.Dropout(0.2)(net)
    out = tf.keras.layers.Dense(1, activation='sigmoid')(net)
    
    model = tf.keras.models.Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(tf.keras.optimizers.Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [20]:
max_len = 100
X_train, X_val, y_train, y_val = sklearn.model_selection.train_test_split(data.text, data['Sarcasm'], 
                                                                          test_size=0.2, random_state=0)
X_train = bert_encode(X_train, tokenizer, max_len=max_len)
X_val = bert_encode(X_val, tokenizer, max_len=max_len)

In [21]:
model = build_model(bert_layer, max_len=max_len)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 100)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 100)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 100)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 109482241   input_word_ids[0][0]             
                                                                 input_mask[0][0]             

In [22]:
%%time
checkpoint = tf.keras.callbacks.ModelCheckpoint('model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, verbose=1)

train_history = model.fit(
    X_train, y_train, 
   
    epochs=10,
    callbacks=[checkpoint, earlystopping],
    batch_size=1,
    verbose=1
)

Epoch 1/10
6382/6382 [==============================] - ETA: 0s - loss: 0.1327 - accuracy: 0.9773WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


6382/6382 [==============================] - 763s 120ms/step - loss: 0.1327 - accuracy: 0.9773
Epoch 2/10
6382/6382 [==============================] - ETA: 0s - loss: 0.1279 - accuracy: 0.9773WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


6382/6382 [==============================] - 762s 119ms/step - loss: 0.1279 - accuracy: 0.9773
Epoch 3/10
6382/6382 [==============================] - ETA: 0s - loss: 0.1229 - accuracy: 0.9773WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


6382/6382 [==============================] - 762s 119ms/step - loss: 0.1229 - accuracy: 0.9773
Epoch 4/10
6382/6382 [==============================] - ETA: 0s - loss: 0.1210 - accuracy: 0.9773WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


6382/6382 [==============================] - 763s 120ms/step - loss: 0.1210 - accuracy: 0.9773
Epoch 5/10
6382/6382 [==============================] - ETA: 0s - loss: 0.1194 - accuracy: 0.9773WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


6382/6382 [==============================] - 761s 119ms/step - loss: 0.1194 - accuracy: 0.9773
Epoch 6/10
6382/6382 [==============================] - ETA: 0s - loss: 0.1191 - accuracy: 0.9773WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


6382/6382 [==============================] - 762s 119ms/step - loss: 0.1191 - accuracy: 0.9773
Epoch 7/10
6382/6382 [==============================] - ETA: 0s - loss: 0.1222 - accuracy: 0.9773WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


6382/6382 [==============================] - 760s 119ms/step - loss: 0.1222 - accuracy: 0.9773
Epoch 8/10
6382/6382 [==============================] - ETA: 0s - loss: 0.1187 - accuracy: 0.9773WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


6382/6382 [==============================] - 759s 119ms/step - loss: 0.1187 - accuracy: 0.9773
Epoch 9/10
6382/6382 [==============================] - ETA: 0s - loss: 0.1156 - accuracy: 0.9773WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


6382/6382 [==============================] - 765s 120ms/step - loss: 0.1156 - accuracy: 0.9773
Epoch 10/10
 427/6382 [=>............................] - ETA: 11:48 - loss: 0.1612 - accuracy: 0.9649Buffered data was truncated after reaching the output size limit.

In [23]:
test_performance=model.evaluate(X_val,y_val)
print("The loss of the model on the test set : {}".format(test_performance[0]))
print("The accuracy of the model on the test set : {}%".format(test_performance[1]*100))



50/50 [==============================] - 19s 380ms/step - loss: 0.0742 - accuracy: 0.9862
The loss of the model on the test set : 0.07417653501033783
The accuracy of the model on the test set : 98.62155318260193%


In [1]:
!git status

fatal: not a git repository (or any of the parent directories): .git
